# WideDeep - TabMLP Hyperparameter Sweep 20211009

In [1]:
# two manual flags (ex-config)
COLAB = False
USE_GPU = True
# libraries = ['xgboost', 'lightgbm', 'catboost']
libraries = ['xgboost', 'lightgbm', 'catboost']

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"sweep_widedeep_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if COLAB:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
#     !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # upgrade sklearn
    !pip install --upgrade scikit-learn

#     !pip install category_encoders
    
    if 'catboost' in libraries:
        !pip install catboost
    
    if 'xgboost' in libraries:
        if USE_GPU: 
            # this part is from https://github.com/rapidsai/gputreeshap/issues/24
            !pip install cmake --upgrade
            # !pip install sklearn --upgrade
            !git clone --recursive https://github.com/dmlc/xgboost
            %cd /content/xgboost
            !mkdir build
            %cd build
            !cmake .. -DUSE_CUDA=ON
            !make -j4
            %cd /content/xgboost/python-package
            !python setup.py install --use-cuda --use-nccl
            !/opt/bin/nvidia-smi
            !pip install shap
        else:
            !pip install --upgrade xgboost
    if 'lightgbm' in libraries:
        if USE_GPU:
            # lighgbm gpu compatible
            !git clone --recursive https://github.com/Microsoft/LightGBM
            ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
        else:
            !pip install --upgrade lightgbm
        

        

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
from sklearn.impute import SimpleImputer #, KNNImputer
# import timm

import seaborn as sns

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler #, MinMaxScaler, MaxAbsScaler, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


In [6]:
from optuna.samplers import TPESampler
import optuna
from sklearn.utils import resample
import sklearn.metrics
from optuna.integration.wandb import WeightsAndBiasesCallback


In [7]:
from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep, SAINT, TabTransformer, TabNet, TabFastFormer, TabResnet
from pytorch_widedeep.metrics import Accuracy
from torchmetrics import AUROC
import torch
from torch.optim import Adam, AdamW, Adagrad, SGD, RMSprop, LBFGS
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts, CyclicLR, OneCycleLR, StepLR, CosineAnnealingLR
from pytorch_widedeep.callbacks import EarlyStopping, LRHistory, ModelCheckpoint

Now, datapath setup

In [8]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
if COLAB:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/oct2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/home/sf/code/kaggle/tabular_playgrounds/oct2021/')
    datapath = root/'datasets'
    edapath = root/'EDA'
    modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    studypath = root/'optuna_studies'
    
    for pth in [root, datapath, edapath, modelpath, predpath, subpath, studypath]:
        pth.mkdir(exist_ok=True)
    


In [10]:
SEED = 42

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(seed=SEED)

## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [11]:
# if exmodel_config['scaler']:
#     scaler = exmodel_config['scaler']()
#     scaler.fit_transform()

In [12]:
train_source = datapath/'train.feather'
df = pd.read_feather(path=train_source)
# df.index.name = 'id'
y = np.array(df.target)
features = [x for x in df.columns if x != 'target']
X = df[features] # passing X as a pd.DataFrame to the trainer below, rather than as an np.ndarray
# X_train = df[features]
# X.index.name = 'id'
# y.index.name = 'id'

In [13]:
# X_train.columns

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# wide_cols = [f for f in X_train.columns if X_train[f].nunique() == 2]

In [15]:
# wide_cols

In [16]:
# X_train.shape

## Ex-Model Config

In [17]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "library": 'widedeep',
    "model": "TabMLP",
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
#     "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
#     "scale_b4_impute": False,
#     "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
#     "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'optuna': True,
    'optuna_trials': 100,
#     'subsample': 1,
#     'cross_val_strategy': None, # None for holdout, or the relevant sklearn class
#     'kfolds': 1, # if 1, that means just doing holdout
#     'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}

wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202110_Kaggle_tabular_playground',
    'tags': ['sweep', 'deep-learning', 'TabMLP'],
    'notes': "Sweep for WideDeep with Optuna",
    'config': exmodel_config,
}

In [18]:
# X = np.array(X_train)
# y = np.array(y_train)

# del df, X_train, y_train


# exmodel_config['feature_count'] = len(X.columns)
exmodel_config['feature_count'] = X.shape[1]
exmodel_config['instance_count'] = X.shape[0]

# exmodel_config['feature_generator'] = None
# exmodel_config['feature_generator'] = "Summary statistics"

exmodel_config['train_source'] = str(train_source)

In [19]:
# X_df = pd.DataFrame(X)

In [20]:
# print(X_df.iloc[:,0])

In [21]:
# print(X_df.iloc[:, list(X_df.columns)[0]])

In [22]:
test_source = datapath/'test.feather'
exmodel_config['test_source'] = str(test_source)
X_test = pd.read_feather(path=test_source)
# X_test = X_test.iloc[:, 1:]

In [23]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [24]:
# X.nunique().sort_values()

# Preprocessing

- Recall that at this point, `X` is a `pd.DataFrame` and `y` is a `np.ndarray`

In [25]:
wide_cols = [f for f in X.columns if X[f].nunique() == 2]
cont_cols = [f for f in X.columns if X[f].nunique() > 2]

exmodel_config['wide_cols'] = str(wide_cols) # str(wide_cols)
exmodel_config['cont_cols'] = str(cont_cols) # str(cont_cols)

In [26]:
wide_preprocessor = WidePreprocessor(wide_cols=wide_cols)
X_wide = wide_preprocessor.fit_transform(X)

In [27]:
tab_preprocessor = TabPreprocessor(continuous_cols=cont_cols)#, embed_cols=embed_cols, )
X_tab = tab_preprocessor.fit_transform(X) 

In [28]:
# X_test_wide = wide_preprocessor.transform(X_test)
# X_test_tab = tab_preprocessor.transform(X_test)

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
# dump(X_wide, datapath/'X_train_wide_original_dataset.joblib')
# dump(X_tab, datapath/'X_train_tab_original_dataset,joblib')
# dump(X_test_wide, datapath/'X_test_wide_original_dataset,joblib')
# dump(X_test_tab, datapath/'X_test_tab_original_dataset.joblib')

In [30]:
# X_wide = load(datapath/'X_train_wide_original_dataset.joblib')
# X_tab = load(datapath/'X_train_tab_original_dataset,joblib')
# X_test_wide = load(datapath/'X_test_wide_original_dataset,joblib')
# X_test_tab = load(datapath/'X_test_tab_original_dataset.joblib')

# Experiment setup

In [40]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial):
    # split the (original Kaggle training) data into partitions
    # if study.best_trial:
    #     print("Dumping best params, which are:")
    #     print(str(study.best_trial.params))
    #     dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
       
    # else:
    #     print("No best study yet")
    X_train_wide, X_valid_wide, y_train, y_valid = train_test_split(X_wide, y, test_size=0.33, random_state=int(SEED), shuffle=True)
    X_train_tab, X_valid_tab, y_train_tab, y_valid_tab = train_test_split(X_tab, y, test_size=0.33, random_state=int(SEED), shuffle=True)
    
#     if (y_train_tab != y_train) or (y_valid_tab != y_valid):
#         print("Splits are not the same")
    
    # create wrappers for the training and validation partitions
    # train_pool = catboost.Pool(X_train, y_train)
    # valid_pool = catboost.Pool(X_valid, y_valid)
    
    # experimental parameters
    
    # to be passed directly to the MLP via splat
    mlp_params = {
        'cont_norm_layer': trial.suggest_categorical('cont_norm_layer', ['batchnorm', 'layernorm', None]),
        'mlp_activation': trial.suggest_categorical('mlp_activation', ['relu', 'tanh', 'leaky_relu', 'gelu']),
        'mlp_batchnorm': trial.suggest_categorical('mlp_batchnorm', [True, False]),
        'mlp_linear_first': trial.suggest_categorical('mlp_linear_first', [True, False]),
    }
    
    # to be passed a la carte to the composite arg (i.e. list)
    mlp_hidden_dim_one = trial.suggest_int('mlp_hidden_dim_one', 64, 400)
    mlp_hidden_dim_two = trial.suggest_int('mlp_hidden_dim_two', 32, 200)
    
    mlp_dropout_layer_one = trial.suggest_uniform('mlp_dropout_layer_one', 0.05, 0.5)
    mlp_dropout_layer_two = trial.suggest_uniform('mlp_dropout_layer_two', 0.05, 0.5)

    wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)
    deeptabular = TabMlp(
        mlp_dropout=[mlp_dropout_layer_one, mlp_dropout_layer_two],
        mlp_hidden_dims=[mlp_hidden_dim_one, mlp_hidden_dim_two],
        column_idx=tab_preprocessor.column_idx,
    #     embed_input=tab_preprocessor.embeddings_input,
        continuous_cols=cont_cols,
        **mlp_params
    )

    # model instantiation and training
    model = WideDeep(wide=wide, deeptabular=deeptabular)
    
#     # optimizers -- optimizer params all default for now
#     wide_opt = trial.suggest_categorical('wide_opt', [
#         torch.optim.Adam(model.wide.parameters()),
#         torch.optim.AdamW(model.wide.parameters()),
#         torch.optim.Adagrad(model.wide.parameters()),
#         torch.optim.SGD(model.wide.parameters()),
#         torch.optim.RMSProp(model.wide.parameters()),
#         torch.optim.LBFGS(model.wide.parameters()),
#     ])
    
#     deep_opt = trial.suggest_categorical('deep_opt', [
#         torch.optim.Adam(model.deep.parameters()),
#         torch.optim.AdamW(model.deep.parameters()),
#         torch.optim.Adagrad(model.deep.parameters()),
#         torch.optim.SGD(model.deep.parameters()),
#         torch.optim.RMSProp(model.deep.parameters()),
#         torch.optim.LBFGS(model.deep.parameters()),
#     ])
    
    # optimizers -- optimizer params all default for now
    wide_opt = trial.suggest_categorical('wide_opt', [Adam, AdamW, Adagrad, SGD, RMSprop, LBFGS])
    
    deep_opt = trial.suggest_categorical('deep_opt', [Adam, AdamW, Adagrad, SGD, RMSprop, LBFGS])

    optimizers = {
        'wide': wide_opt(model.wide.parameters()),
        'deep': deep_opt(model.deeptabular.parameters()), # note this is deeptabular, NOT deep (as in some old examples)
    }
    
    # schedulers
    # required params
#     step_size = trial.suggest_int('lr_step_size', 3, 7)
#     base_lr = trial.suggest_uniform('base_lr', 0.01, 0.1)
#     max_lr = trial.suggest_uniform('max_lr', 0.1, 0.3)
    
    # leaving params default for now, except for required ones
    wide_sch = trial.suggest_categorical('wide_sch', [StepLR, ReduceLROnPlateau, CosineAnnealingLR, CosineAnnealingWarmRestarts, CyclicLR, OneCycleLR])
    
    
#         torch.optim.lr_scheduler.StepLR(wide_opt, step_size=5),
#         torch.optim.lr_scheduler.ReduceLROnPlateau(wide_opt),
#         torch.optim.lr_scheduler.CosineAnnealingLR(wide_opt),
#         torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(wide_opt, T_0=3),
#         torch.optim.lr_scheduler.CyclicLR(wide_opt, base_lr=0.01, max_lr=0.3),
#         torch.optim.lr_scheduler.OneCycleLR(wide_opt, max_lr=0.3),
#     ])
    
    deep_sch = trial.suggest_categorical('deep_sch', [StepLR, ReduceLROnPlateau, CosineAnnealingLR, CosineAnnealingWarmRestarts, CyclicLR, OneCycleLR])
    
#         torch.optim.lr_scheduler.StepLR(deep_opt, step_size=3),
#         torch.optim.lr_scheduler.ReduceLROnPlateau(deep_opt),
#         torch.optim.lr_scheduler.CosineAnnealingLR(deep_opt),
#         torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(deep_opt, T_0=3),
#         torch.optim.lr_scheduler.CyclicLR(deep_opt, base_lr=0.01, max_lr=0.3),
#         torch.optim.lr_scheduler.OneCycleLR(deep_opt, max_lr=0.3),
#     ])
    
    # defining dict containing the schedulers -- still need to supply args
    schedulers = {
        'wide': wide_sch,
        'deep': deep_sch
    }
    
    # pass the appropriate arguments, depending on the schedulers' interfaces
    for modality in schedulers.keys():
        if schedulers[modality] == StepLR:
            if modality == 'wide':
                schedulers[modality] = schedulers[modality](optimizers[modality], step_size=5)
            else: # deep
                schedulers[modality] = schedulers[modality](optimizers[modality], step_size=3)
        elif schedulers[modality] == CosineAnnealingLR:
            schedulers[modality] - schedulers[modality](optimizers[modality], T_max=10)
        elif schedulers[modality] == CosineAnnealingWarmRestarts:
            schedulers[modality] = schedulers[modality](optimizers[modality], T_0=3)
        elif schedulers[modality] == CyclicLR:
            schedulers[modality] = schedulers[modality](optimizers[modality],base_lr=0.01, max_lr=0.3)
        elif schedulers[modality] == OneCycleLR:
            schedulers[modality] = schedulers[modality](optimizers[modality],base_lr=0.01)
        else:
            schedulers[modality] = schedulers[modality](optimizers[modality])
                
                
#     if StepLR in [wide_sch, deep_sch]:
#         step_size_wide = 5
#         step_size_deep = 3
        
#     if CosineAnnealingWarmRestarts in [wide_sch, deep_sch]:
#         T_0 = 3
        
#     if CyclicLR in [wide_sch, deep_sch] or OneCycleLR in [wide_sch, deep_sch]:
#         base_lr = 0.01
#         max_lr = 0.3
#         wide_sch_with_args = 
    
    # to be passed via splat to the Trainer
#     trainer_params = {
#         'optimizers': optimizers,
#         'lr_schedulers': trial.suggest_categorical('lr_schedulers', [ReduceLROnPlateau, CyclicLR, OneCycleLR, CosineAnnealingLR, CosineAnnealingWarmRestarts, StepLR]),
#         'initializers': trial.suggest_categorical('initializers' #),
#     }
        

    # to be passed to the Trainer.fit call
    n_epochs = trial.suggest_int('n_epochs', 5, 100)

    trainer = Trainer(model=model, 
                      objective='binary', 
                      metrics=[Accuracy, AUROC], # with AUROC got TypeError: '>' not supported between instances of 'NoneType' and 'int' 
                      seed=SEED, 
                      optimizers=optimizers,
                      lr_schedulers=schedulers, # note the lack of lr in the identifier
                      callbacks=[
                          LRHistory(n_epochs=n_epochs), 
                          ModelCheckpoint(
#                               filepath=modelpath/f"{exmodel_config['library']}_{exmodel_config['model']}_rs{42}_best-model_{datetime.now().strftime('%Y%m%d%H%M%S')}.joblib",
                              filepath='/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/optuna_widedeep_tabmlp_20211010_best.pth',
                              save_best_only=True,
                          ),
                      ],
                     )

#             print(f"type(X_train_wide) is {type(X_train_wide)} and type(X_train_tab) is {type(X_train_tab)}")

    
    trainer.fit( # this is where problem is beginning
        X_wide=X_train_wide,
        X_tab=X_train_tab,
        target=y_train,
        n_epochs=n_epochs,
        batch_size=1024, # default value is 32

#         initializers= # pending
    )
    
    # generate predictions
    train_preds = trainer.predict_proba(X_wide=X_train_wide, X_tab=X_train_tab, batch_size=1024)
    valid_preds = trainer.predict_proba(X_wide=X_valid_wide, X_tab=X_valid_tab, batch_size=1024)[:,1]
    # rounds to the nearest integer, and the nearest even in case of _.5s

    # Evaluation
    train_auc = roc_auc_score(y_true=y_train, y_score=train_preds)
    valid_auc = roc_auc_score(y_true=y_valid, y_score=valid_preds)
    print('ROC AUC Score on Training Set of TabMLP model =', train_auc)
    print('ROC AUC Score on Validation Set of TabMLP model =', valid_auc)
    wandb.log({'valid_auc': valid_auc,})

    return valid_auc

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
type(torch.optim.lr_scheduler.StepLR)

type

In [33]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-33-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find sweep_widedeep_20211010.ipynb
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [34]:
study = optuna.create_study(direction = "maximize", 
                            sampler = TPESampler(seed=int(SEED)), 
                            study_name=f"widedeep_TabMLP_{datetime.now().strftime('%Y%m%d')}")

# study = load(datapath/f'optuna_lightgbm_26trials-complete_20211002.joblib')


[I 2021-10-10 09:40:31,224] A new study created in memory with name: widedeep_TabMLP_20211010


In [41]:
for x in range(1,101):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc],) #catch=(lightgbm.basic.LightGBMError,)) #n_jobs = multiprocessing.cpu_count())
#     print(f"{x+26} trials complete")
    dump(study, filename=studypath/f"optuna_{exmodel_config['library']}-{exmodel_config['model']}_study_{x}trials_{datetime.now().strftime('%Y%m%d')}.joblib")
#     dump(study.best_trial.params, filename=datapath/f'optuna_lightgbm_study_best-thru-{x*5}trials_20210927.joblib')

/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adam.Adam'> which is of type type.
  warnings.warn(message)
/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adamw.AdamW'> which is of type type.
  warnings.warn(message)
/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adagrad.Adagrad'> which is of type type.
  warnings.warn(message)
/home/sf/anaconda3/envs/tabular-x/lib/python3.8/site-packag

ValueError: optimizer must support momentum with `cycle_momentum` option enabled

In [ ]:
# dump(study, filename=datapath/f"optuna_lightgbm_26trials-complete_{datetime.now().strftime('%Y%m%d')}.joblib")
# dump(study.best_trial.params, filename=datapath/f"optuna_lightgbm_all-500trials-best_{datetime.now().strftime('%Y%m%d')}.joblib")
# pickle.dump(study.best_trial.params, open('CatBoost_Hyperparameter.pickle', 'wb'))
# print('CatBoost Hyperparameter:', study.best_trial.params)

In [ ]:
study.best_trial.params

In [ ]:
wandb.log({'optuna_best_params': study.best_trial.params})
wandb.finish()

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)